In [ ]:
import os
import pyvista as pv

pv.set_jupyter_backend('static')
os.environ['DISPLAY'] = ':99.0'
os.environ['PYVISTA_OFF_SCREEM'] = 'true'

In [ ]:
is_export = True
off_screen = True

mesh_path = '/mnt/d/knpob/4-data/20211229-DynaBreast4D/3dmd/6kmh_27marker_2/meshes'
landmark_path = '/mnt/d/knpob/4-data/20211229-DynaBreast4D/vicon/6kmh_27marker_2'

start=0
stride = 12
end=120

export_folder = "output/visual/"

# Load data

In [ ]:
from mesh4d import obj3d

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=mesh_path,
    start=start,
    stride=stride,
    end=end,
)

In [ ]:
import numpy as np
from mesh4d import kps, utils

vicon_arr = np.load(os.path.join(landmark_path, 'vicon_arr.npy'))
vicon_start = utils.load_pkl_object(os.path.join(landmark_path, 'vicon_start.pkl'))
vicon_cab = utils.load_pkl_object(os.path.join(landmark_path, 'vicon>>3dmd.pkl'))

landmarks = kps.MarkerSet()
landmarks.load_from_array(vicon_arr, start_time=vicon_start, fps=100, trans_cab=vicon_cab)
landmarks.interp_field()

In [ ]:
from mesh4d.analyse.crave import clip_with_contour

contour = landmarks.extract((0, 1, 10, 17, 25, 26))
mesh_clip_ls = clip_with_contour(mesh_ls, start_time=0, fps=120/stride, contour=contour, clip_bound='xy', margin=30)

In [ ]:
body_ls = obj3d.init_obj_series(
    mesh_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
breast_ls = obj3d.init_obj_series(
    mesh_clip_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
from mesh4d import obj4d

body4d = obj4d.Obj4d_Deform(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=False,
)
body4d.add_obj(*body_ls)
body4d.load_markerset('landmarks', landmarks)

In [ ]:
breast4d = obj4d.Obj4d_Deform(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=False,
)
breast4d.add_obj(*breast_ls)
breast4d.load_markerset('landmarks', landmarks)

# Visualization

In [ ]:
if is_export:
    body4d.animate(export_folder=export_folder, filename='body4d')

In [ ]:
body4d.show(elements='m', off_screen=off_screen, is_export=is_export, export_folder=export_folder, export_name='body4d')

In [ ]:
body4d.show(elements='mk', off_screen=off_screen, is_export=is_export, export_folder=export_folder, export_name='body4d_kps')

In [ ]:
if is_export:
    breast4d.animate(export_folder=export_folder, filename='breast4d')
    breast4d.animate(export_folder=export_folder, filename='breast4d_raw', elements='m')

In [ ]:
import pyvista as pv

stack_dist = 1000
zoom_rate = 3.5
window_size = [2000, 800]

scene = pv.Plotter(off_screen=off_screen)
plot_num = len(body4d.obj_ls)

for idx in range(0, plot_num):
    body = body4d.obj_ls[idx]
    breast = breast4d.obj_ls[idx]
    width = body.get_width()

    body.add_mesh_to_scene(scene, location=[0, 0, idx*stack_dist], opacity=0.1)
    breast.add_mesh_to_scene(scene, location=[0, 0, idx*stack_dist])
    breast.add_kps_to_scene(scene, location=[0, 0, idx*stack_dist], radius=0.02*width)
    
scene.camera_position = 'zy'
scene.camera.azimuth = 45
scene.camera.zoom(zoom_rate)
scene.window_size = window_size
scene.enable_parallel_projection()
# scene.show(interactive_update=True)
scene.show()

if is_export:
    export_path = os.path.join(export_folder, 'crop_stack.png')
    scene.update()
    scene.screenshot(export_path)